In [2]:
%load_ext autoreload
%autoreload 2

from dataloader import WICO
import tensorflow as tf
from model import Net
from spektral.data import DisjointLoader
from tensorflow import keras
from tqdm import tqdm
import numpy as np

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy

In [3]:
def train_step(model, loss_fn, optimizer, inputs, target):

    with tf.GradientTape() as tape:
        predictions, _, _ = model(inputs[:2])  # drop edge features from inputs
        loss = loss_fn(target, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc

In [5]:
def random_split(data, train_pct=0.75, train_epochs=5, seed=None):
    """ returns two DataLoaders, resp. for train and test set 
        according to the input split percentage """
    if seed is not None:
        np.random.default_rng(seed)

    np.random.shuffle(data)
    n = len(data)
    train_size = int(n*train_pct)

    train_set = data[:train_size]
    test_set = data[train_size:]

    train_loader = DisjointLoader(train_set, batch_size=1, epochs=train_epochs)
    test_loader = DisjointLoader(test_set, batch_size=1, epochs=1)

    return train_loader, test_loader

In [8]:
def evaluate(train_loader, test_loader):
    model = Net(num_classes=3)
    loss_fn = CategoricalCrossentropy(from_logits=True)
    optimizer = Adam(learning_rate=0.001)

    epoch = step = 0
    train_results = []

    for inputs, target in tqdm(train_loader):
        step += 1
        loss, acc = train_step(model, loss_fn, optimizer, inputs, target)
        train_results.append((loss, acc))

        if step == train_loader.steps_per_epoch:
            step = 0
            epoch += 1
            print("Ep. {} - Loss: {}. Acc: {}".format(epoch, *np.mean(train_results, 0)))
            train_results = []
    
    test_results = []

    for inputs, target in test_loader:
        predictions, _, _ = model(inputs[:2], training=False)
        test_results.append(
            (
                loss_fn(target, predictions),
                tf.reduce_mean(categorical_accuracy(target, predictions)),
            )
        )
    
    print("Done. Test loss: {}. Test acc: {}".format(*np.mean(test_results, 0)))

    return train_results, test_results

In [12]:
data = WICO("./dataset/WICO/", root_edges=False, time_delay_edges=False)

crossval_results = []
n_splits = 5

for i in range(n_splits):
    print(f"Experiment {i+1} of {n_splits}")
    train_loader, test_loader = random_split(data, train_epochs=3)

    train_results, test_results = evaluate(train_loader, test_loader)

    crossval_results.append((train_results, test_results))

Experiment 1 of 5


2619it [14:45,  2.30it/s]

Ep. 1 - Loss: 0.8032735586166382. Acc: 0.7163039445877075


5238it [29:26,  2.35it/s]

Ep. 2 - Loss: 0.796207845211029. Acc: 0.7163039445877075


...

5238it [31:39,  2.12it/s]

Ep. 2 - Loss: 0.7876783013343811. Acc: 0.7212676405906677


7857it [47:34,  2.75it/s]

Ep. 3 - Loss: 0.7880550622940063. Acc: 0.7212676405906677


Done. Test loss: 0.8338673114776611. Test acc: 0.6884307265281677
